## csr_matrix ?
- 모델링 직전에

In [99]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor 

In [592]:
train = pd.read_csv("./data/train.csv")

In [901]:
test = pd.read_csv("./data/test.csv")

In [144]:
samplesub = pd.read_csv("./data/sample_submission.csv")

In [644]:
samplesub.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preprocessing

## Check Train 'NaN' data before Imputation

In [593]:
train_desc = train.describe(include='all').transpose()
train_desc['MV_count'] = train.isnull().sum()
train_desc['MV_ratio'] = train_desc['MV_count']/train.shape[0]
train_desc

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,MV_count,MV_ratio
TripType,647054,NaN,NaN,NaN,58.5845,157.636,3,27,39,40,999,0,0.000000
VisitNumber,647054,NaN,NaN,NaN,96167.6,55545.5,5,49268,97074,144316,191347,0,0.000000
Weekday,647054,7,Sunday,133975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
Upc,642925,NaN,NaN,NaN,3.0607e+10,9.12013e+10,834,3.4e+09,7.0501e+09,3.00653e+10,9.78971e+11,4129,0.006381
ScanCount,647054,NaN,NaN,NaN,1.10888,0.700776,-12,1,1,1,71,0,0.000000
DepartmentDescription,645693,68,GROCERY DRY GOODS,70402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1361,0.002103
FinelineNumber,642925,NaN,NaN,NaN,3726.88,2780.97,0,1404,3352,5501,9998,4129,0.006381


## Check Test 'NaN' data before Imputation

In [582]:
test_desc = test.describe(include='all').transpose()
test_desc['MV_count'] = test.isnull().sum()
test_desc['MV_ratio'] = test_desc['MV_count']/test.shape[0]
test_desc

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,MV_count,MV_ratio
VisitNumber,653646,NaN,NaN,NaN,97041,55575.2,1,49689,98219,145271,191348,0,0.000000
Weekday,653646,7,Sunday,133233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
Upc,649660,NaN,NaN,NaN,3.05432e+10,9.12545e+10,834,3.4e+09,7.04705e+09,3.00417e+10,9.78971e+11,3986,0.006098
ScanCount,653646,NaN,NaN,NaN,1.10873,0.712841,-21,1,1,1,55,0,0.000000
DepartmentDescription,652318,67,GROCERY DRY GOODS,72335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1328,0.002032
FinelineNumber,649660,NaN,NaN,NaN,3725.24,2776.63,0,1404,3353,5501,9999,3986,0.006098


## Preprocessing Step1 - Imputation 
아래 순서처럼 대분류, 중분류, 소분류 순으로 Imputation 한다.
- DepartmentDescription 
- FinelineNumber
- Upc

### 1-1. Train - DepartmentDescription

- VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움

In [594]:
# 일단 DepartmentDescription에 'MENSWEAR' 와 'MENS WEAR'가 구분되어 있던 것을 'MENS WEAR'로 통일
train.loc[train['DepartmentDescription']=='MENSWEAR','DepartmentDescription']='MENS WEAR'
train[train['DepartmentDescription']=='MENSWEAR']

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [595]:
# before imputation
train['DepartmentDescription'].isna().sum()

1361

In [596]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = train[train['DepartmentDescription'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(train[train["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index) != 0:
        train.loc[(train["VisitNumber"] == idx)&(train["DepartmentDescription"].isna()), "DepartmentDescription"] \
        = train[train["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index[0]

100%|██████████| 1172/1172 [00:37<00:00, 30.98it/s]


In [597]:
# after imputation step1
train['DepartmentDescription'].isna().sum()

191

In [598]:
# 아직 처리하지 못한 총 191개 데이터는 VisitNumber 따른 DepartmentDescription 값이 모두 비어있는 경우인데
# 이 경우 TripType이 모두 999이 때문에 그런 DepartmentDescription는 TripType 999의 DepartmentDescription 최빈값으로 imputation 함.
train.loc[train["DepartmentDescription"].isna(), "DepartmentDescription"] \
= train[train["TripType"] == 999]["DepartmentDescription"].value_counts().index[0]

In [599]:
train[train["TripType"] == 999]["DepartmentDescription"].value_counts().index[0]

'FINANCIAL SERVICES'

In [600]:
# after imputation step2
train['DepartmentDescription'].isna().sum()

0

### 1-2. Test - DepartmentDescription

- VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움

In [902]:
# 일단 DepartmentDescription에 'MENSWEAR' 와 'MENS WEAR'가 구분되어 있던 것을 'MENS WEAR'로 통일
test.loc[test['DepartmentDescription']=='MENSWEAR','DepartmentDescription']='MENS WEAR'
test[test['DepartmentDescription']=='MENSWEAR']

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [903]:
# before imputation
test['DepartmentDescription'].isna().sum()

1328

In [904]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = test[test['DepartmentDescription'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(test[test["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index) != 0:
        test.loc[(test["VisitNumber"] == idx)&(test["DepartmentDescription"].isna()), "DepartmentDescription"] \
        = test[test["VisitNumber"] == idx]["DepartmentDescription"].value_counts().index[0]

100%|██████████| 1141/1141 [00:36<00:00, 30.90it/s]


In [905]:
# after imputation step1
test['DepartmentDescription'].isna().sum()

219

In [906]:
# 아직 처리하지 못한 총 219개 데이터는 VisitNumber 따른 DepartmentDescription 값이 모두 비어있는 경우인데
# 이 경우 Train데이터에서 TripType이 모두 999이었고 그런 DepartmentDescription의 최빈값인 'FINANCIAL SERVICES'로 imputaion 함.
test.loc[test["DepartmentDescription"].isna(), "DepartmentDescription"] = 'FINANCIAL SERVICES'

In [907]:
train[train["TripType"] == 999]["DepartmentDescription"].value_counts().index[0]

'FINANCIAL SERVICES'

In [908]:
# after imputation step2
test['DepartmentDescription'].isna().sum()

0

### 2-1. Train - FinelineNumber

- VisitNumber에 따른 FinelineNumber의 최빈값으로 FinelineNumber의 NaN 값을 채움

In [602]:
# before imputation
train['FinelineNumber'].isna().sum()

4129

In [603]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = train[train['FinelineNumber'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(train[train["VisitNumber"] == idx]["FinelineNumber"].value_counts().index) != 0:
        train.loc[(train["VisitNumber"] == idx)&(train["FinelineNumber"].isna()), "FinelineNumber"] \
        = train[train["VisitNumber"] == idx]["FinelineNumber"].value_counts().index[0]

100%|██████████| 2754/2754 [00:15<00:00, 172.27it/s]


In [604]:
# after imputation step1
train['FinelineNumber'].isna().sum()

2369

In [605]:
# 아직 처리하지 못한 총 2369개 데이터는 TripType으로 구분해보면 5와 999 두 개로 나뉜다.
train[train['FinelineNumber'].isna()]['TripType'].value_counts()

5      2089
999     280
Name: TripType, dtype: int64

In [606]:
train_after_step1_FL_null_VM_list = train[train['FinelineNumber'].isna()]['VisitNumber'].value_counts().index
train_after_step1_FL_null_VM_list

Int64Index([ 74003,  52567, 111813, 114942, 156445,  83995,  41497,  79927,
              1860,  69775,
            ...
            168331,  78201, 127351,  78197, 168307,  16747,  14684, 102743,
              2390,  16384],
           dtype='int64', length=1427)

In [607]:
# 따라서 각 TripType의 최빈 FinelineNumber 값으로 imputation 함.
train.loc[(train["TripType"] == 5)&(train["FinelineNumber"].isna()), "FinelineNumber"] \
= train[train["TripType"] == 5]["FinelineNumber"].value_counts().index[0]
train.loc[(train["TripType"] == 999)&(train["FinelineNumber"].isna()), "FinelineNumber"] \
= train[train["TripType"] == 999]["FinelineNumber"].value_counts().index[0]

In [608]:
train[train["TripType"] == 5]["FinelineNumber"].value_counts().index[0]

5501.0

In [609]:
train[train["TripType"] == 999]["FinelineNumber"].value_counts().index[0]

279.0

In [610]:
# after imputation step2
train['FinelineNumber'].isna().sum()

0

### 2-2. Test - FinelineNumber

- VisitNumber에 따른 FinelineNumber의 최빈값으로 FinelineNumber의 NaN 값을 채움

In [909]:
# before imputation
test['FinelineNumber'].isna().sum()

3986

In [910]:
# DepartmentDescription이 NaN 값인 데이터의 VisitNumber 리스트
DD_null_VM = test[test['FinelineNumber'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 DepartmentDescription의 최빈값으로 DepartmentDescription의 NaN 값을 채움
for idx in tqdm(DD_null_VM):
    if len(test[test["VisitNumber"] == idx]["FinelineNumber"].value_counts().index) != 0:
        test.loc[(test["VisitNumber"] == idx)&(test["FinelineNumber"].isna()), "FinelineNumber"] \
        = test[test["VisitNumber"] == idx]["FinelineNumber"].value_counts().index[0]

100%|██████████| 2706/2706 [00:18<00:00, 150.32it/s]


In [911]:
# after imputation step1
test['FinelineNumber'].isna().sum()

2257

In [912]:
# 아직 처리하지 못한 총 2257개 데이터는 VisitNumber 따른 FinelineNumber 값이 모두 비어있는 경우인데
# 이 경우 Train데이터에서 TripType이 5 혹은 999이어서 그에 맞춰 FinelineNumber의 최빈값을 imputation할 수 있었지만
# Test데이터는 TripType을 기준으로 imputation 할 수 없고 train데이터와 VisitNumber와 도 겹치지 않기 때문에
# Train데이터와 Test데이터의 Weekday, ScanCount, DepartmentDescription을 기준으로 FinelineNumber를 imputation 함.

In [913]:
# train데이터와 test데이터의 imputation step1 이후의 VisitNumber가 겹치지 않음
test_after_step1_FL_null_VM_list = test[test['FinelineNumber'].isna()]['VisitNumber'].value_counts().index
[i for i in test_after_step1_FL_null_VM_list if i in train_after_step1_FL_null_VM_list]

[]

In [914]:
#1 : Sunday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Sunday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Sunday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Sunday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Sunday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0]),
            'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

146
2111


In [915]:
#2 : Sunday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Sunday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Sunday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Sunday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Sunday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

12
2099


In [916]:
#3 : Sunday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Sunday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Sunday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Sunday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType']==train.loc[(train['Weekday']=='Sunday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

17
2082


In [917]:
#4 : Sunday, -1, PHARMACY RX 
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Sunday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Sunday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Sunday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType']==train.loc[(train['Weekday']=='Sunday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0]
         ),'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
2080


In [918]:
#5 : Sunday, 2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Sunday')&(test['ScanCount']==2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Sunday')
     &(test['ScanCount']==2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Sunday')
        &(train['ScanCount']==2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType']==train.loc[(train['Weekday']=='Sunday')
                                         &(train['ScanCount']==2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0]
         ),'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
2078


In [919]:
#6 : Saturday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType']==train.loc[(train['Weekday']=='Saturday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0]
         ),'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

28
2050


In [920]:
#7 : Saturday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType']==train.loc[(train['Weekday']=='Saturday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0]
         ),'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

294
1756


In [921]:
#8 : Saturday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Saturday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

14
1742


In [922]:
#9 : Friday, -1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Friday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Friday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Friday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Friday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

7
1735


In [923]:
#10 : Friday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Friday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Friday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Friday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Friday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

376
1359


In [924]:
#11 : Friday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Friday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Friday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Friday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Friday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

19
1340


In [925]:
#12 : Friday, 2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Friday')&(test['ScanCount']==2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Friday')
     &(test['ScanCount']==2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Friday')
        &(train['ScanCount']==2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Friday')
                                         &(train['ScanCount']==2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
1339


In [926]:
#13 : Friday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Friday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Friday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Friday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Friday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

5
1334


In [927]:
#14 : Thursday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Thursday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Thursday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Thursday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Thursday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

301
1033


In [928]:
#15 : Thursday, -1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Thursday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Thursday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Thursday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Thursday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

6
1027


In [929]:
#16 : Thursday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Thursday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Thursday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Thursday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Thursday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

5
1022


In [930]:
#17 : Wednesday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

12
1010


In [931]:
#18 : Wednesday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

329
681


In [932]:
#19 : Wednesday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

19
662


In [933]:
#20 : Wednesday, -1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

24
638


In [934]:
#21 : Wednesday, 2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
                                         &(train['ScanCount']==2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
637


In [935]:
#22 : Tuesday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

297
340


In [936]:
#23 : Tuesday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

13
327


In [937]:
#24 : Monday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Monday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Monday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Monday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Monday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

19
308


In [938]:
#25 : Monday, 1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Monday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Monday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Monday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Monday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

244
64


In [939]:
#26 : Monday, 1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Monday')&(test['ScanCount']==1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Monday')
     &(test['ScanCount']==1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Monday')
        &(train['ScanCount']==1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Monday')
                                         &(train['ScanCount']==1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

11
53


In [940]:
#27 : Saturday, -2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==-2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==-2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
        &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Saturday')
                                         &(train['ScanCount']==-2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

3
50


In [941]:
#28 : Saturday, 2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
        &(train['ScanCount']==2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Saturday')
                                         &(train['ScanCount']==2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
48


In [942]:
#29 : Thursday, 2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Thursday')&(test['ScanCount']==2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Thursday')
     &(test['ScanCount']==2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Thursday')
        &(train['ScanCount']==2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Thursday')
                                         &(train['ScanCount']==2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
47


In [943]:
#30 : Thursday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Thursday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Thursday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Thursday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Thursday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

8
39


In [944]:
#31 : Wednesday, -2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==-2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==-2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
                                         &(train['ScanCount']==-2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

4
35


In [945]:
#33 : Tuesday, -1, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

12
23


In [946]:
#34 : Tuesday, -1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
        &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
                                         &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

4
19


In [947]:
#35 : Tuesday, 2, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==2)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==2)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
        &(train['ScanCount']==2)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
                                         &(train['ScanCount']==2)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
17


In [948]:
#37 : Monday, -1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Monday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Monday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Monday')
#         &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Monday')
#                                          &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
15


In [949]:
#38 : Saturday, -1, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==-1)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==-1)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
#         &(train['ScanCount']==-1)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Saturday')
#                                          &(train['ScanCount']==-1)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
13


In [950]:
#39 : Saturday, -5, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Saturday')&(test['ScanCount']==-5)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Saturday')
     &(test['ScanCount']==-5)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Saturday')
#         &(train['ScanCount']==-5)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Saturday')
#                                          &(train['ScanCount']==-5)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
12


In [951]:
#40 : Monday, -2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Monday')&(test['ScanCount']==-2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Monday')
     &(test['ScanCount']==-2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Monday')
#         &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Monday')
#                                          &(train['ScanCount']==-2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
10


In [952]:
#41 : Friday, -2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Friday')&(test['ScanCount']==-2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Friday')
     &(test['ScanCount']==-2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Friday')
#         &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Friday')
#                                          &(train['ScanCount']==-2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
9


In [953]:
#42 : Tuesday, -2, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==-2)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==-2)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
#         &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
#                                          &(train['ScanCount']==-2)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
8


In [954]:
#43 : Sunday, -4, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Sunday')&(test['ScanCount']==-4)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Sunday')
     &(test['ScanCount']==-4)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Sunday')
#         &(train['ScanCount']==-4)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Sunday')
#                                          &(train['ScanCount']==-4)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
7


In [955]:
train.loc[(train['Weekday']=='Monday')
        &(train['ScanCount']==-3)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [956]:
#44 : Monday, -3, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Monday')&(test['ScanCount']==-3)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Monday')
     &(test['ScanCount']==-3)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Monday')
#         &(train['ScanCount']==-3)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Monday')
#                                          &(train['ScanCount']==-3)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

3
4


In [957]:
train.loc[(train['Weekday']=='Wednesday')
        &(train['ScanCount']==-5)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [958]:
#45 : Wednesday, -5, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Wednesday')&(test['ScanCount']==-5)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Wednesday')
     &(test['ScanCount']==-5)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Wednesday')
#         &(train['ScanCount']==-5)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Wednesday')
#                                          &(train['ScanCount']==-5)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
3


In [959]:
train.loc[(train['Weekday']=='Tuesday')
        &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='PHARMACY RX')]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [960]:
#46 : Tuesday, -2, PHARMACY RX
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Tuesday')&(test['ScanCount']==-2)
                   &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Tuesday')
     &(test['ScanCount']==-2)
     &(test['DepartmentDescription']=='PHARMACY RX'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Tuesday')
#         &(train['ScanCount']==-2)
        &(train['DepartmentDescription']=='PHARMACY RX')
        &(train['TripType'] == train.loc[(train['Weekday']=='Tuesday')
#                                          &(train['ScanCount']==-2)
                                         &(train['DepartmentDescription']=='PHARMACY RX'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

2
1


In [961]:
#47 : Thursday, -3, FINANCIAL SERVICES
print(len(test.loc[(test['FinelineNumber'].isna())&(test['Weekday']=='Thursday')&(test['ScanCount']==-3)
                   &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']))
test.loc[(test['FinelineNumber'].isna())
     &(test['Weekday']=='Thursday')
     &(test['ScanCount']==-3)
     &(test['DepartmentDescription']=='FINANCIAL SERVICES'),'FinelineNumber']\
= train.loc[(train['Weekday']=='Thursday')
#         &(train['ScanCount']==-3)
        &(train['DepartmentDescription']=='FINANCIAL SERVICES')
        &(train['TripType'] == train.loc[(train['Weekday']=='Thursday')
#                                          &(train['ScanCount']==-3)
                                         &(train['DepartmentDescription']=='FINANCIAL SERVICES'),
                                         'TripType'].value_counts().index[0])
            ,'FinelineNumber'].value_counts().index[0]
print(test['FinelineNumber'].isna().sum())

1
0


In [962]:
test[test['FinelineNumber'].isna()]

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber


In [963]:
# after imputation step2
test['FinelineNumber'].isna().sum()

0

### 3-1. Train - Upc

- VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움

In [971]:
# before imputation
train['Upc'].isna().sum()

4129

In [972]:
# Upc이 NaN 값인 데이터의 VisitNumber 리스트
Upc_null_VM = train[train['Upc'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움
for idx in tqdm(Upc_null_VM):
    if len(train[train["VisitNumber"] == idx]["Upc"].value_counts().index) != 0:
        train.loc[(train["VisitNumber"] == idx)&(train["Upc"].isna()), "Upc"] \
        = train[train["VisitNumber"] == idx]["Upc"].value_counts().index[0]

100%|██████████| 2754/2754 [00:17<00:00, 161.69it/s]


In [973]:
# after imputation step1
train['Upc'].isna().sum()

2369

In [974]:
# 아직 처리하지 못한 총 2369개 데이터는 TripType으로 구분해보면 5와 999 두 개로 나뉜다.
train[train['Upc'].isna()]['TripType'].value_counts()

5      2089
999     280
Name: TripType, dtype: int64

In [975]:
train_after_step1_Upc_null_VM_list = train[train['Upc'].isna()]['VisitNumber'].value_counts().index
train_after_step1_Upc_null_VM_list

Int64Index([ 74003,  52567, 111813, 114942, 156445,  83995,  41497,  79927,
              1860,  69775,
            ...
            168331,  78201, 127351,  78197, 168307,  16747,  14684, 102743,
              2390,  16384],
           dtype='int64', length=1427)

In [992]:
train.loc[(train['DepartmentDescription']=='FINANCIAL SERVICES')
         &(train['FinelineNumber']==279.0)]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
509,999,253,Friday,6.811318e+10,-1,FINANCIAL SERVICES,279.0
590,999,281,Friday,6.811318e+10,-1,FINANCIAL SERVICES,279.0
709,999,317,Friday,6.053881e+10,-1,FINANCIAL SERVICES,279.0
949,999,403,Friday,6.053881e+10,-1,FINANCIAL SERVICES,279.0
950,999,407,Friday,6.811318e+10,-1,FINANCIAL SERVICES,279.0
959,999,409,Friday,NaN,-1,FINANCIAL SERVICES,279.0
1134,999,484,Friday,NaN,-2,FINANCIAL SERVICES,279.0
1135,999,484,Friday,NaN,-2,FINANCIAL SERVICES,279.0
1357,999,583,Friday,6.053881e+10,-1,FINANCIAL SERVICES,279.0
2041,999,904,Friday,6.053881e+10,-1,FINANCIAL SERVICES,279.0


In [991]:
train.loc[(train['DepartmentDescription']=='FINANCIAL SERVICES')
         &(train['FinelineNumber']==279.0)
        ,'Upc'].value_counts()

6.053881e+10    510
6.811318e+10    323
6.053881e+10     22
Name: Upc, dtype: int64

In [987]:
train[(train['Upc'].isna())
     &(train['TripType']==999)]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
959,999,409,Friday,NaN,-1,FINANCIAL SERVICES,279.0
1134,999,484,Friday,NaN,-2,FINANCIAL SERVICES,279.0
1135,999,484,Friday,NaN,-2,FINANCIAL SERVICES,279.0
6285,999,2245,Friday,NaN,-1,FINANCIAL SERVICES,279.0
8524,999,3004,Friday,NaN,1,FINANCIAL SERVICES,279.0
9094,999,3149,Friday,NaN,1,FINANCIAL SERVICES,279.0
14917,999,4884,Friday,NaN,1,FINANCIAL SERVICES,279.0
21546,999,6836,Saturday,NaN,-1,FINANCIAL SERVICES,279.0
21693,999,6885,Saturday,NaN,-1,FINANCIAL SERVICES,279.0
28029,999,8681,Saturday,NaN,-1,FINANCIAL SERVICES,279.0


In [981]:
train[train["TripType"] == 5]["Upc"].value_counts().index[0]

4011.0

In [541]:
# 따라서 각 TripType의 최빈 Upc 값으로 imputation 함.
train.loc[(train["TripType"] == 5)&(train["Upc"].isna()), "Upc"] \
= train[train["TripType"] == 5]["Upc"].value_counts().index[0]
train.loc[(train["TripType"] == 999)&(train["Upc"].isna()), "Upc"] \
= train[train["TripType"] == 999]["Upc"].value_counts().index[0]

In [542]:
train[train["TripType"] == 5]["Upc"].value_counts().index[0]

4011.0

In [543]:
train[train["TripType"] == 999]["Upc"].value_counts().index[0]

60538812238.0

In [544]:
# after imputation step2
train['Upc'].isna().sum()

0

### 3-2. Test - Upc

- VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움

In [964]:
# before imputation
test['Upc'].isna().sum()

3986

In [965]:
# Upc이 NaN 값인 데이터의 VisitNumber 리스트
Upc_null_VM = test[test['Upc'].isna()]['VisitNumber'].unique()
# VisitNumber에 따른 Upc의 최빈값으로 Upc의 NaN 값을 채움
for idx in tqdm(Upc_null_VM):
    if len(test[test["VisitNumber"] == idx]["Upc"].value_counts().index) != 0:
        test.loc[(test["VisitNumber"] == idx)&(test["Upc"].isna()), "Upc"] \
        = test[test["VisitNumber"] == idx]["Upc"].value_counts().index[0]

100%|██████████| 2706/2706 [00:17<00:00, 154.66it/s]


In [966]:
# after imputation step1
test['Upc'].isna().sum()

2257

In [967]:
# 아직 처리하지 못한 총 2257개 데이터는 VisitNumber 따른 Upc 값이 모두 비어있는 경우인데
# 이 경우 Train데이터에서 TripType이 5 혹은 999이어서 그에 맞춰 Upc 최빈값을 imputation할 수 있었지만
# Test데이터는 TripType을 기준으로 imputation 할 수 없고 train데이터와 VisitNumber와 도 겹치지 않기 때문에
# Train데이터와 Test데이터의 Weekday, ScanCount, DepartmentDescription을 기준으로 Upc를 imputation 함.

In [968]:
# train데이터와 test데이터의 imputation step1 이후의 VisitNumber가 겹치지 않음
test_after_step1_Upc_null_VM_list = test[test['Upc'].isna()]['VisitNumber'].value_counts().index
[i for i in test_after_step1_Upc_null_VM_list if i in train_after_step1_Upc_null_VM_list]

[]

In [969]:
# after imputation step2
test['Upc'].isna().sum()

2257

## Check Train 'NaN' data after Imputation

In [970]:
desc = train.describe(include='all').transpose()
desc['MV_count'] = train.isnull().sum()
desc['MV_ratio'] = desc['MV_count']/train.shape[0]
desc

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,MV_count,MV_ratio
TripType,647054,NaN,NaN,NaN,58.5845,157.636,3,27,39,40,999,0,0.000000
VisitNumber,647054,NaN,NaN,NaN,96167.6,55545.5,5,49268,97074,144316,191347,0,0.000000
Weekday,647054,7,Sunday,133975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
Upc,642925,NaN,NaN,NaN,3.0607e+10,9.12013e+10,834,3.4e+09,7.0501e+09,3.00653e+10,9.78971e+11,4129,0.006381
ScanCount,647054,NaN,NaN,NaN,1.10888,0.700776,-12,1,1,1,71,0,0.000000
DepartmentDescription,647054,67,GROCERY DRY GOODS,70530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000
FinelineNumber,647054,NaN,NaN,NaN,3731,2778.86,0,1404,3353,5501,9998,0,0.000000


## Check Test 'NaN' data after Imputation

In [268]:
desc = train.describe(include='all').transpose()
desc['MV_count'] = train.isnull().sum()
desc['MV_ratio'] = desc['MV_count']/train.shape[0]
desc

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,MV_count,MV_ratio
TripType,647054,NaN,NaN,NaN,58.5845,157.636,3,27,39,40,999,0,0.0
VisitNumber,647054,NaN,NaN,NaN,96167.6,55545.5,5,49268,97074,144316,191347,0,0.0
Weekday,647054,7,Sunday,133975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
Upc,647054,NaN,NaN,NaN,3.05455e+10,9.10702e+10,834,3.4e+09,7.04705e+09,3.0065e+10,9.78971e+11,0,0.0
ScanCount,647054,NaN,NaN,NaN,1.10888,0.700776,-12,1,1,1,71,0,0.0
DepartmentDescription,647054,67,GROCERY DRY GOODS,70530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
FinelineNumber,647054,NaN,NaN,NaN,3731,2778.86,0,1404,3353,5501,9998,0,0.0


## Preprocessing Step2 -  Encoding

### Weekday

In [12]:
# before
train['Weekday'].value_counts()

Sunday       133975
Saturday     122096
Friday        96247
Monday        83130
Tuesday       72529
Wednesday     71115
Thursday      67962
Name: Weekday, dtype: int64

In [16]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

train['Weekday'] = weekday_preprocessing(train)
train['Weekday'].value_counts()

1    133975
2    122096
3     96247
4     83130
5     72529
6     71115
7     67962
Name: Weekday, dtype: int64

In [20]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,1,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,1,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,1,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,1,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,1,3.800060e+09,1,GROCERY DRY GOODS,3600.0


### ScanCount

- VisitNumber로 groupby 할때 sum으로 대체

In [19]:
# after preprocessing
desc['MV_count'] = train.isnull().sum()
desc['MV_ratio'] = desc['MV_count']/train.shape[0]
desc

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,MV_count,MV_ratio
TripType,647054,NaN,NaN,NaN,58.5845,157.636,3,27,39,40,999,0,0.000000
VisitNumber,647054,NaN,NaN,NaN,96167.6,55545.5,5,49268,97074,144316,191347,0,0.000000
Weekday,647054,NaN,NaN,NaN,3.4997,2.00343,1,2,3,5,7,0,0.000000
Upc,642925,NaN,NaN,NaN,3.0607e+10,9.12013e+10,834,3.4e+09,7.0501e+09,3.00653e+10,9.78971e+11,4129,0.006381
ScanCount,647054,NaN,NaN,NaN,1.10888,0.700776,-12,1,1,1,71,0,0.000000
DepartmentDescription,645693,68,GROCERY DRY GOODS,70402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1361,0.002103
FinelineNumber,642925,NaN,NaN,NaN,3726.88,2780.97,0,1404,3352,5501,9998,4129,0.006381


##  Feature importance via Gradient Boosting model

In [52]:
X = train.drop(columns='TripType')
y = train['TripType']

In [54]:
start_time=time.time()

gb = GradientBoostingRegressor()
gb.fit(X, y)
features = X.columns.values

end_time=time.time()
print("total time in the current cell ",end_time-start_time,"s")

ValueError: could not convert string to float: 'GROCERY DRY GOODS'

In [46]:
# Scatter plot 
trace = go.Scatter(
    y = gb.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        color = gb.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Machine Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)